In [23]:
from skimage import feature
import cv2
import os
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score
from sklearn import datasets
import joblib
from sklearn.model_selection import cross_val_score

label_group = ['city','nature','old']
features_list = []
labels_list = []

for i in range(0,3):
        # Get image features of city images
        read_path = 'dataset_1/'+label_group[i]
        for filename in os.listdir(read_path):
                if filename.lower().endswith(('.png', '.jpg', '.jpeg')):
                        image_path = read_path + "/" + filename
                        image = cv2.imread(image_path,cv2.IMREAD_GRAYSCALE)
                        fd = feature.hog(image, orientations=9, pixels_per_cell=(8,8),
                                                cells_per_block=(2, 2), visualize=False)
                        features_list.append(fd)
                        labels_list.append(label_group[i])
                
# Turn features and labels into NumPy array
features = np.array(features_list)
labels = np.array(labels_list)
state = np.random.get_state()
np.random.shuffle(features)
np.random.set_state(state)
np.random.shuffle(labels)


# Save the array as .npz file
np.savez('hog_old_features_and_labels.npz', features=features, labels=labels)






In [24]:
if __name__ == "__main__":
    npz_file = "hog_old_features_and_labels.npz"

    # Load npz file
    loaded_data = np.load(npz_file)

    # Show keys in the file
    print("Keys in the npz file:", list(loaded_data.keys()))

    # Load features and labels array
    features = loaded_data['features']
    labels = loaded_data['labels']

    # Show features and labels size
    print("Features shape:", features.shape)
    print("Labels shape:", labels.shape)

    # Show one feature vector and its label
    print("Feature vector:")
    print(features[5])
    print("Label:")
    print(labels[5])


Keys in the npz file: ['features', 'labels']
Features shape: (257, 142884)
Labels shape: (257,)
Feature vector:
[0.23609903 0.23018251 0.10052838 ... 0.24299501 0.12131064 0.09769914]
Label:
nature


In [25]:
if __name__ == "__main__":
    npz_file = "hog_old_features_and_labels.npz"

    # Load npz file
    loaded_data = np.load(npz_file)

    # Load features and labels array
    features = loaded_data['features']
    labels = loaded_data['labels']

    X = features
    y = labels

    # Create SVM model
    svm_classifier = SVC(kernel='linear', C=1.0, random_state=43)
    
    # Cross validation
    scores = cross_val_score(svm_classifier, X, y, cv=5)
    print("Scores:", scores)
    
    # Divide train and test dataset
    X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.2, random_state=43)
    svm_classifier.fit(X_train, y_train)

    # Save model as a file
    model_filename = "hog_old_svm_model.joblib"
    joblib.dump(svm_classifier, model_filename)

    # Load saved model
    loaded_model = joblib.load(model_filename)

    # Predict using loaded model
    y_pred_loaded = loaded_model.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred_loaded)
    print("Accuracy:", accuracy)

Scores: [0.80769231 0.82692308 0.74509804 0.78431373 0.74509804]
Accuracy: 0.7307692307692307
